# Plot ERE obtained by Lüscher formalism from database 

## Init

In [ ]:
import os
from itertools import product

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pylab as plt

from luescher_nd.database.utilities import read_table

In [ ]:
import matplotlib
import re

markers = re.findall('``"([a-z]+)"``', matplotlib.markers.__doc__)

In [ ]:
%load_ext blackcellmagic

In [ ]:
DATA = os.path.join(os.getcwd(), os.pardir, "data")

In [ ]:
!ls $DATA

## Plotting

Load the data frame

In [ ]:
DB_NAME = "db-contact-fv-c-fitted-parity-a-inv.sqlite"

df = read_table(os.path.join(DATA, DB_NAME), dispersion_zeta=False, round_digits=1)
df.head()

Specify grid properties

In [ ]:
row = "$L$ [fm$^{-1}$]"
hue = "epsilon"
col = "nstep"
col_order = df.nstep.unique()
x = "$p^2$ [fm$^{-2}$]"
y = "$p \cot(\delta_0(p))$ [fm$^{-1}$]"

Specify data ranges

In [ ]:
query = "x > -5"

and plot

In [ ]:
tf = df.query(query).sort_values("x")
for key, val in {
    "y": "$p \cot(\delta_0(p))$ [fm$^{-1}$]",
    "L": "$L$ [fm$^{-1}$]",
    "p2": "$p^2$ [fm$^{-2}$]",
}.items():
    tf[val] = tf[key]

grid = sns.FacetGrid(
    data=tf,
    col=col,
    hue=hue,
    row=row,
    sharex="row",
    sharey=True,
    legend_out=True,
    hue_kws={"marker": markers},
    palette="cubehelix",
    margin_titles=True,
    col_order=col_order
)

grid.map(plt.plot, x, y, ms=3, lw=0.5, ls="--", marker="s")

for ax in grid.axes.flatten():
    #ax.set_yscale("log")
    ax.set_ylim(-6, 1)

grid.fig.set_dpi(200)
grid.add_legend(frameon=False)

plt.show()


# Effective for different dispersion relations

Compare EREs for different fits and ERE methods

In [ ]:
DB_NAME = "db-contact-fv-c-fitted-parity-a-inv.sqlite"

df_c = read_table(os.path.join(DATA, DB_NAME), dispersion_zeta=False, round_digits=1)
df_d = read_table(os.path.join(DATA, DB_NAME.replace("fv-c", "fv-d")), dispersion_zeta=True, round_digits=1)
df_cd = read_table(os.path.join(DATA, DB_NAME), dispersion_zeta=True, round_digits=1)

df_c["type"] = "Regular Lüscher"
df_d["type"] = "Dispersion Lüscher"
df_cd["type"] = "RL fit, DL ERE"

df = df_c.append(df_d, ignore_index=True)
df = df.append(df_cd, ignore_index=True)


In [ ]:
df.head()

In [ ]:
row = "$L$ [fm$^{-1}$]"
hue = "epsilon"
col = "type"
x = "$p^2$ [fm$^{-2}$]"
y = "$p \cot(\delta_0(p))$ [fm$^{-1}$]"

In [ ]:
query = "nstep == '$\\infty$' and L == 1"

tf = df.query(query).sort_values("x")
for key, val in {
    "y": "$p \cot(\delta_0(p))$ [fm$^{-1}$]",
    "L": "$L$ [fm$^{-1}$]",
    "p2": "$p^2$ [fm$^{-2}$]",
}.items():
    tf[val] = tf[key]

grid = sns.FacetGrid(
    data=tf,
    col=col,
    hue=hue,
    #row=row,
    sharex="row",
    sharey=True,
    legend_out=True,
    hue_kws={"marker": markers},
    palette="cubehelix",
    margin_titles=False,
    row_order=df["type"].unique()
)

grid.map(plt.plot, x, y, ms=3, lw=0.5, ls="--", marker="s")

for ax in grid.axes.flatten():
    #ax.set_yscale("log")
    ax.set_ylim(-6, 1)

grid.fig.suptitle(query, y=1.05)
    
grid.fig.set_dpi(200)
grid.add_legend(frameon=False)

plt.show()

# Continuum extrapolation of ERE error

Plot the difference of the analytic ERE ($-\frac{1}{a_0}$) and the numerically extracted ERE coming from Lüschers formulation for different excited states and discretizations.
The numerical ERE was extracted by fits against spherical Lüscher energy levels.
Only pick discretizations which show no dispersion effects (straight line in ERE(x) plot).

In [ ]:
DB_NAME = "db-contact-fv-c-fitted-parity-a-inv.sqlite"

df = read_table(os.path.join(DATA, DB_NAME), dispersion_zeta=False, round_digits=1)
df.head()

In [ ]:
count_frame = df.groupby(["L", "nstep", "nlevel"]).count()["epsilon"]
drop_ids = count_frame[count_frame < 4].index

tf = (
    df.set_index(["L", "nstep", "nlevel"])
    .drop(drop_ids)
    .query("nlevel > 0 and nstep != '1'")
    .reset_index()
    .sort_values("epsilon")
    .copy()
)
tf["diff"] = (tf["y"] + 5.0).abs()

grid = sns.FacetGrid(
    data=tf,
    col="L",
    hue="nlevel",
    row="nstep",
    sharex="col",
    sharey=True,
    legend_out=True,
    # hue_kws={"marker": markers},
    palette="cubehelix",
    margin_titles=True,
)

grid.map(plt.plot, "epsilon", "diff", ms=3, lw=0.5, ls="--", marker="o")
grid.add_legend(title= "Energy level", bbox_to_anchor=(1.1,0.5))
grid.fig.set_dpi(300)
grid.fig.set_figheight(4)
grid.set_ylabels(r"$\left| p \cot(\delta_0(p)) + \frac{1}{a_0} \right|$ [fm$^{-1}$]")
grid.set_xlabels(r"$\epsilon$ [fm]")


for ax in grid.axes.flatten():
    ax.set_yscale("log")
    ax.set_xscale("log")
    # ax.set_ylim(1.e-2, 1.e2)
